In [ ]:
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from scipy import linalg
import scipy.stats as stats
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt 
import seaborn as sns 

%matplotlib inline

In [ ]:
bhdata=datasets.load_boston()

In [ ]:
featuredata=bhdata.data

In [ ]:
bhdata.keys()

In [ ]:
print(bhdata.DESCR)

In [ ]:
colnames=bhdata.feature_names
colnames

In [ ]:
data = pd.DataFrame(featuredata,columns=colnames)

In [ ]:
data['target']=bhdata.target

In [ ]:
data.head(20)

In [ ]:
#Data Preprocessing

In [ ]:
data.isnull().sum()

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.distplot(data['target'], bins=30)
plt.show()

In [ ]:
corr= data.corr().round(2)
corr

In [ ]:
plt.figure(figsize=(14, 10))
# annot = True to print the values inside the square
ax=sns.heatmap(data=corr, annot=True)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.title("Correlation Matrix")
plt.show()

In [ ]:
plt.figure(figsize=(20, 5))

features = ['LSTAT', 'RM']
target = data['target']

for i, col in enumerate(features):
    plt.subplot(1, len(features) , i+1)
    x = data[col]
    y = target
    plt.scatter(x, y, marker='o')
    plt.title(col)
    plt.xlabel(col)
    plt.ylabel('Median Housing Price')

In [ ]:
#How can we look at all the data at one.
#Pairwise correlation plot

sns.pairplot(data)

In [ ]:
sns.lmplot(x = 'RM', y = 'target', data = data)

In [ ]:
#SUmmary Statistics
data.describe().T

In [ ]:
#split the dataset into Train and Test dataset

In [ ]:
data_train, data_test = train_test_split(data, test_size=0.20, shuffle=True,random_state=1234)

In [ ]:
data_train.head()

In [ ]:
print(data_train.shape,data_test.shape)

In [ ]:
data_test.head()

In [ ]:
XTrain = data_train.iloc[:,0:(data_train.shape[1]-1)].values
XTrain

In [ ]:
YTrain=data_train['target']

In [ ]:
XTest = data_test.iloc[:,0:(data_test.shape[1]-1)].values

In [ ]:
YTest=data_test['target']

In [ ]:
XTrain.shape, XTest.shape

In [ ]:
#Multiple Linear Regression Model
reg= linear_model.LinearRegression(normalize=True)

In [ ]:
?LinearRegression

In [ ]:
reg.fit(XTrain, YTrain)

In [ ]:
print("Coefficients: \n", reg.coef_)

In [ ]:
reg.score(XTrain, YTrain)

In [ ]:
colnames

In [ ]:
reg.singular_

In [ ]:
XTrain

In [ ]:
?LinearRegression

In [ ]:
pd.DataFrame(zip(colnames,reg.coef_),columns=['Input_features','Model_Coefficients'])

In [ ]:
reg.intercept_

In [ ]:
# predicting the test set results 
Y_pred = reg.predict(XTest) 

In [ ]:
# Plotting Scatter graph to show the prediction  
# results - 'y_actual' value vs 'y_pred' value 
plt.scatter(YTest, Y_pred, c = 'green') 
plt.xlabel("Actual Housing Price: in $1000's") 
plt.ylabel("Predicted Housing Price: in $1000's") 
plt.title("Actual value vs predicted value : Linear Regression") 
plt.show() 

In [ ]:
#Adjusted R-Square
def AdjustedRSquare(model,X,Y):
    YHat= model.predict(X)
    n,k = X.shape
    sse = np.sum(np.square(YHat-Y),axis=0)
    sst = np.sum(np.square(Y-np.mean(Y)),axis=0)
    R2 = 1 - sse/sst
    adjR2 = R2 - (1- R2)*(float(k)/(n-k-1))
    return adjR2, R2

In [ ]:
#https://www.xspdf.com/help/50077476.html
#Return P-Value
def ReturnPValue(model,X,Y):
    YHat= model.predict(X)
    n,k = X.shape
    sse = np.sum(np.square(YHat-Y),axis=0)
    x = np.hstack((np.ones((n,1)),np.matrix(X)))
    df = float(n-k-1)
    sampleVar = sse/df
    sampleVarianceX = x.T*x
    covarianceMatrix = linalg.sqrtm(sampleVar*sampleVarianceX.I)
    se = covarianceMatrix.diagonal()[1:]
    betasTstat = np.zeros(len(se))
    for i in range(len(se)):
        betasTstat[i] = model.coef_[i]/se[i]
    betasPvalue = 1- stats.t.cdf(abs(betasTstat),df)
    return betasPvalue

In [ ]:
reg.BetasPValue=ReturnPValue(reg,XTrain,YTrain)
reg.BetasPValue

In [ ]:
reg.adjR2, reg.R2 = AdjustedRSquare(reg,XTrain,YTrain)
print(reg.adjR2, reg.R2)

In [ ]:
from sklearn import linear_model
import statsmodels.api as sm
from scipy import stats

X = XTrain
y = YTrain

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

In [ ]:
# Null : Beta estimates = 0
# Alt : Beta Estimates != 0

In [ ]:
print("Coefficients: \n", reg.coef_)

In [ ]:
p_values = est2.pvalues
p_values

In [ ]:
#Two methods of reporting the error from the regression model
#Mean Absolute Percentage Error (MAPE)
#Mean Sum of Square error (MSSE)

In [ ]:
def ErrorMetric(model,X,Y):
    Yhat = model.predict(X)
    MAPE = np.mean(abs(Y-Yhat)/Y)*100
    MSSE = np.mean(np.square(Y-Yhat))
    return MAPE, MSSE

In [ ]:
reg.TrainMAPE, reg.TrainMSSE = ErrorMetric(reg,XTrain,YTrain)
reg.TrainMAPE, reg.TrainMSSE

In [ ]:
reg.MAPE, reg.MSSE = ErrorMetric(reg,XTest,YTest)
reg.MAPE, reg.MSSE

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

print('MAE', mean_absolute_error(YTest, Y_pred))
print('MSE', mean_squared_error(YTest, Y_pred))
print('RMSE', np.sqrt(mean_squared_error(YTest, Y_pred)))
print('R squared error', r2_score(YTest, Y_pred))

In [ ]:
sns.distplot((YTest-Y_pred),bins=50)

In [ ]:
# Check for multi Collinearity
# Coefficient estimates may not be reliable
# Consider removing variables with a high Variance Inflation Factor (VIF)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
print('Variance Inflation Factors (VIF)')
print('> 10: An indication that multicollinearity may be present')
print('> 100: Certain multicollinearity among the variables')
print('-------------------------------------')
       
# Gathering the VIF for each variable
VIF = [variance_inflation_factor(featuredata, i) for i in range(featuredata.shape[1])]
for idx, vif in enumerate(VIF):
    print('{0}: {1}'.format(colnames[idx], vif))

In [ ]:
# Test for homoscedasticity, which is the same variance within our error terms
# Heteroscedasticity means we don’t have an even variance across the error terms.
# Significance tests for coefficients due to the standard errors being biased. 
# Additionally, the confidence intervals will be either too wide or too narrow.

In [ ]:
residuals = abs(YTest)-abs(Y_pred)
residuals

In [ ]:
# Plotting the residuals
plt.subplots(figsize=(12, 6))
ax = plt.subplot(111)  # To remove spines
plt.scatter(x=residuals.index, y=residuals, alpha=0.5)
plt.plot(np.repeat(0, residuals.index.max()), color='darkorange', linestyle='--')
ax.spines['right'].set_visible(False)  # Removing the right spine
ax.spines['top'].set_visible(False)  # Removing the top spine
plt.title('Residuals')
plt.show()  

In [ ]:
# There don’t appear to be any obvious problems with that and variance appears to be uniform across the error terms.

In [ ]:
# Existence of outliers inflates the B coefficients, It has the capacity to destabilize the estimated statistics
# and may represent incorrect R2 and Adj R2

# Outliers can be managed by removing them or replacing them with various interpolation methods
# It can be replaced by median values/mean values/percentile based approach/caping based approach

In [ ]:
data.DIS.describe()

In [ ]:
sns.boxplot(x=data['DIS'])

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
ax.scatter(data['INDUS'], data['TAX'])
ax.set_xlabel('Proportion of non-retail business acres per town')
ax.set_ylabel('Full-value property-tax rate per $10,000')
plt.show()

In [ ]:
for column in data:
    plt.figure()
    data.boxplot([column])

In [ ]:
data.plot(kind='box')

In [ ]:
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
((data < (Q1 - 3 * IQR)) | (data > (Q3 + 3 * IQR))).sum()

In [ ]:
data_df_out = data[~((data < (Q1 - 1.5 * IQR)) |(data > (Q3 + 1.5 * IQR))).any(axis=1)]
data_df_out.shape

In [ ]:
data.shape

In [ ]:
# Z-score of an observation is a distance from the mean in the units of standard deviation
from scipy import stats
z = np.abs(stats.zscore(data))
print(z)

In [ ]:
z.shape

In [ ]:
pd.DataFrame(z).head(20)

In [ ]:
threshold = 3
print(np.where(z > threshold))
z[np.where(z > threshold)]

In [ ]:
print(z[55][1])

In [ ]:
data_o = data[(z < 3).all(axis=1)]
data_o.shape

In [ ]:
#If I drop redundant variables, then it will significantly improve the model accuracy
# Stepwise feature selection is an iterative method to identify the best features
# Helps in removing High multi colliearity among the predictors
# the Variable selection for the predictive model can happen in three different ways:
# Forward Selection of Variables
# Backward Selection of Variables
# Both

In [ ]:
# Forward Selection:
# Null model, add another variable and continue to do that until it is adding up the r-square values

In [ ]:
# Base Model Accuracy
# ADj R2 -- 0.7158605022207648 
# R2 --- 0.7250262924717079 
# MAPE - 16.909381848539258
# MSSE - 23.964571384956862

In [ ]:
XTrain.shape, XTest.shape

In [ ]:
alpha = 0.05
erroR = 1

In [ ]:
prevMAPE, prevMSSE= ErrorMetric(reg,XTest,YTest)
print(prevMAPE, prevMSSE)

In [ ]:
#prevadjR2, prevR2 = AdjustedRSquare(reg,XTrain,YTrain)
#prevadjR2, prevR2 

In [ ]:
while (erroR > 0.001):
    ind = np.where(reg.BetasPValue > alpha)[0]
    if(len(ind)==0):
        ind = np.argmax(reg.BetasPValue)
    
    XTrain = np.delete(XTrain, ind, axis=1)
    XTest = np.delete(XTest, ind, axis= 1)
    
    reg.fit(XTrain, YTrain)
    
    reg.BetasPValue = ReturnPValue(reg,XTrain,YTrain)
    MAPE, MSSE = ErrorMetric(reg,XTest,YTest)
    print(MSSE)
    #adjR2, R2 = AdjustedRSquare(reg,XTrain,YTrain)
    
    erroR = prevMSSE - MSSE
    #erroR = adjR2 - prevadjR2
    
print(erroR,MSSE)
#print(erroR,adjR2)

In [ ]:
reg.coef_

In [ ]:
reg.BetasPValue

In [ ]:
XTrain.shape

In [ ]:
pd.DataFrame(XTrain).head()

## Dummy Variables

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.RAD.unique()

In [ ]:
data.RAD.value_counts()

In [ ]:
RAD_Dummies = pd.get_dummies(data.RAD,prefix="RAD").iloc[:,0:]

In [ ]:
RAD_Dummies.head()

In [ ]:
# for K levels, we will create K-1 dummy variables
# If you have 4 categories, then create three dummy variables

In [ ]:
#Dropped the first column
RAD_Dummies = pd.get_dummies(data.RAD,prefix="RAD").iloc[:,1:]
RAD_Dummies

In [ ]:
modeldata=pd.concat([data,RAD_Dummies],axis=1)
modeldata.head()

In [ ]:
modeldata.shape

In [ ]:
modeldata_df=modeldata.iloc[:,list(range(8)) + list(range(9,modeldata.shape[1]))]
modeldata_df.head()

In [ ]:
modeldata_df.shape

In [ ]:
#Alternative approach
modeldata_df1=pd.get_dummies(data, columns=['RAD'],drop_first=True)
modeldata_df1.head()

In [ ]:
modeldata_df1.shape

In [ ]:
data.CHAS.unique()

In [ ]:
data.CHAS.value_counts()

In [ ]:
modeldata_df2=pd.get_dummies(data, columns=['RAD','CHAS'],drop_first=True)
modeldata_df2.head()

In [ ]:
modeldata_df2.shape

In [ ]:
df_train, df_test = train_test_split(modeldata_df, test_size=0.20, shuffle=True,random_state=1234)

In [ ]:
df_train.shape,df_test.shape

In [ ]:
#Splitting into further XTrain, YTrain
df_train.columns

In [ ]:
XTrain = df_train.iloc[:,list(range(12))+list(range(13,df_train.shape[1]))].values

In [ ]:
YTrain = df_train['target']
YTrain

In [ ]:
XTrain.shape,YTrain.shape

In [ ]:
XTest = df_test.iloc[:,list(range(12))+list(range(13,df_test.shape[1]))].values
YTest = df_test['target']

In [ ]:
XTest.shape,YTest.shape

In [ ]:
#Fitting the dataset with Dummy variables
regr = linear_model.LinearRegression(normalize=True)
regr

In [ ]:
regr.fit(XTrain,YTrain)

In [ ]:
regr.score(XTrain,YTrain)

In [ ]:
regr.MAPE, regr.MSSE = ErrorMetric(regr,XTest,YTest)

In [ ]:
regr.MAPE, regr.MSSE

In [ ]:
regr.adjR2, regr.R2 = AdjustedRSquare(regr,XTrain,YTrain)

In [ ]:
regr.adjR2, regr.R2

In [ ]:
# Base Model Accuracy
# ADj R2 -- 0.7158605022207648 
# R2 --- 0.7250262924717079 
# MAPE - 16.909381848539258
# MSSE - 23.964571384956862

# After Creation of dummy variables-- Model Accuracy
# ADj R2 -- 0.7241554031130855 
# R2 --- 0.737844961271245 
# MAPE - 17.16019696950829
# MSSE - 24.949060732511533

In [ ]:
#Since there is not much improvement in accuracy with inclusion of dummy variable
#It is important to explore the existence of non linear relationship

#Polynomial features --> Polynomial Regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

In [ ]:
featuredata

In [ ]:
pd.DataFrame(featuredata).head()

In [ ]:
featuredata.shape

In [ ]:
scaler = StandardScaler()

In [ ]:
X = scaler.fit_transform(featuredata)
X

In [ ]:
X.shape

In [ ]:
?StandardScaler

In [ ]:
#Need to pass the degree of the polynomial. Here we are generating polynomial features of degree =2
poly = PolynomialFeatures(2)

In [ ]:
Xpoly = poly.fit_transform(X)

In [ ]:
Xpoly.shape

In [ ]:
pd.DataFrame(Xpoly).head()

In [ ]:
#Visualise the Polynomial feature names
df1 = pd.DataFrame(featuredata,columns=colnames)
df1.head()

In [ ]:
df1.shape

In [ ]:
poly.fit_transform(df1)
polynomial_feature_names=poly.get_feature_names(df1.columns)
polynomial_feature_names

In [ ]:
len(polynomial_feature_names)

In [ ]:
Xpoly.shape

In [ ]:
Y = bhdata.target
Y.shape

In [ ]:
#Fitting Linear regression model on the polynomial features
regr2 = linear_model.LinearRegression(normalize=True)

In [ ]:
regr2.fit(Xpoly,Y)

In [ ]:
regr2.score(Xpoly,Y)

In [ ]:
regr2.MAPE, regr2.MSSE = ErrorMetric(regr2,Xpoly,Y)
regr2.MAPE, regr2.MSSE

In [ ]:
regr2.adjR2, regr2.R2 = AdjustedRSquare(regr2,Xpoly,Y)
regr2.adjR2, regr2.R2

In [ ]:
# Base Model Accuracy
# ADj R2 -- 0.7158605022207648 
# R2 --- 0.7250262924717079 
# MAPE - 16.909381848539258
# MSSE - 23.964571384956862

# After Inclusion of dummy variables
# ADj R2 -- 0.7241554031130855 
# R2 --- 0.737844961271245 
# MAPE - 17.16019696950829
# MSSE - 24.949060732511533

# After Inclusion of Polynomial Features
# ADj R2 -- 0.9103557309584304 
# R2 --- 0.9289946383829152 
# MAPE - 9.39831696764503
# MSSE - 5.994241112422332

In [ ]:
regr2.coef_

In [ ]:
# How do we generalize the model results of the polynomial model
# Overfitting is something we need to address - Whenever model given unusually high accuracy
# Sometimes fitting large number of features can also lead to overfitting of model

In [ ]:
# To restrict model overfitting, we perform regularisation
# Regularisation penalises the coeff with large values and helps us in avoiding large coefficients in a predictive model
# Regularisation helps us to produces a model which is more generalised 
# Regularised regression models 

In [ ]:
# LASSO(least absolute shrinkage and selection operator) - L1 Regularisation
# Ridge - L2 Regularisation

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
Xpoly.shape, Y.shape

In [ ]:
XTrain, XTest, YTrain, Ytest = train_test_split(Xpoly, Y, test_size=0.25, shuffle=True,random_state=1234)

In [ ]:
XTrain.shape, YTrain.shape, XTest.shape, Ytest.shape

In [ ]:
# alpha => Regularised Parameter
lasso = Lasso(alpha=2)
lasso

In [ ]:
lasso.fit(XTrain,YTrain)

In [ ]:
lasso.coef_

In [ ]:
# Unimportant feature's Beta estimates are compressed to 0
names = [polynomial_feature_names[i] for i in list(np.where(lasso.coef_ != 0.0)[0])]

In [ ]:
names

In [ ]:
lasso.MAPE, lasso.MSSE = ErrorMetric(lasso,XTest,Ytest)
lasso.MAPE, lasso.MSSE

In [ ]:
lasso.adjR2, lasso.R2 = AdjustedRSquare(lasso,XTrain,YTrain)
lasso.adjR2, lasso.R2

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
ridge = Ridge(alpha=5)

In [ ]:
ridge.fit(XTrain,YTrain)

In [ ]:
ridge.coef_

In [ ]:
ridge.MAPE, ridge.MSSE = ErrorMetric(ridge,XTest,Ytest)
ridge.MAPE, ridge.MSSE

In [ ]:
ridge.adjR2, ridge.R2 = AdjustedRSquare(ridge,XTrain,YTrain)
ridge.adjR2, ridge.R2

In [ ]:
# Choosing the correct Alpha is very important
# How to get the optimum value of Alpha
# Grid Search

In [ ]:
from scipy.stats import uniform as sp_rand
from sklearn.model_selection import RandomizedSearchCV
# prepare a uniform distribution to sample for the alpha parameter
param_grid = {'alpha': sp_rand()}
# create and fit a ridge regression model, testing random alpha values
model = Ridge()
rsearch = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=100)
rsearch.fit(bhdata.data, bhdata.target)
print(rsearch)
# summarize the results of the random parameter search
print(rsearch.best_score_)
print(rsearch.best_estimator_.alpha)

In [ ]:
rsearch.best_params_

In [ ]:
#By Considering the Best estimator for alpha
ridge1 = Ridge(alpha=0.99321)

ridge1.fit(XTrain,YTrain)
ridge1.coef_

In [ ]:
ridge1.MAPE, ridge1.MSSE = ErrorMetric(ridge1,XTest,Ytest)
ridge1.MAPE, ridge1.MSSE

In [ ]:
ridge1.adjR2, ridge1.R2 = AdjustedRSquare(ridge1,XTrain,YTrain)
ridge1.adjR2, ridge1.R2